# 라이브러리 불러오기

In [1]:
import sys
sys.path.append('src')
from en_system_ex_analysis.En_system_intergrated import *
from SALib.sample import saltelli
from SALib.analyze import sobol
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import dartwork_mpl as dm
import en_system_ex_analysis as cu
import seaborn as sns
from scipy.stats import norm, uniform, triang, gamma
import warnings
from pprint import pprint
warnings.filterwarnings("ignore")
import matplotlib.colors as mcolors
from matplotlib.cm import get_cmap
from matplotlib import cm
from matplotlib.patches import Patch
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.integrate import quad
from scipy.special import erfc
from scipy.integrate import dblquad
from matplotlib.ticker import LogLocator, LogFormatterMathtext
import concurrent.futures

dm.use_dmpl_style()


Load colors...
Load colormaps...


# 파라미터 값 지정

In [2]:
T_g   = 15        # initial ground temperature, °C
T_0   = 30        # environmental temperature, °C 
q     = 25        # heat flow rate per unit length, W/m
H     = 200       # total length of the GHE, m
k_g   = 2.0       # effective borehole thermal conductivity, W/(m·K)
c_g   = 800       # specific heat capacity of ground, J/(kg·K)
rho_g = 2000      # density of ground, kg/m³
alpha = k_g / (c_g * rho_g)    # thermal diffusivity of the ground, m²/s
r_b   = 0.08      # borehole radius, m
R_b   = 0.108     # effective borehole thermal resistance, (m·K)/W
c_f   = 4184      # specific heat of fluid, J/(kg·K)
rho_f = 1000      # density of fluid, kg/m³
V_f   = 0.0004     # volumetric flow rate of fluid, m³/s
D = 0

E_pmp = 200      # pump energy consumption, W

# 함수 지정 및 리스트 생성

In [3]:
def g_function(t):
    def integrand(z, h):
        term1 = np.sqrt(r_b**2 + (z - h)**2)
        term2 = np.sqrt(r_b**2 + (z + h)**2)
        val1 = erfc(term1 / (2 * np.sqrt(alpha * t))) / term1
        val2 = erfc(term2 / (2 * np.sqrt(alpha * t))) / term2
        return val1 - val2

    # t에 따라 정밀도 조절
    if t < 100*cu.h2s:
        epsabs, epsrel = 1e-300, 1e-300
    else:
        epsabs, epsrel = 1e-8, 1e-8

    # 적분 수행
    integral1, _ = dblquad(
        integrand,
        D, D + H,                 # h limits
        lambda z: D,              # z lower limit
        lambda z: D + H,          # z upper limit
        epsabs=epsabs,            # 정밀도 설정
        epsrel=epsrel
    )

    return integral1 / (4 * np.pi * k_g * H)

def Tf(t):
    return T_g + q * g_function(t) + q * R_b

def Tb(t):
    return T_g + q * g_function(t)


In [4]:
times_hour = np.logspace(0, 5, 200)
times_sec = times_hour * 3600

Tf_o_now = T_g

# --- 결과 저장 리스트 ---
Tf_i_list, Tf_o_list, Tf_list, Tb_list = [], [], [], []
X_in_ghe, X_c_ghe, X_out_ghe = [], [], []
X_in_ground, X_c_ground, X_out_ground = [], [], []
R_g_list = []

def compute_at_time(t):
    R_g = g_function(t)
    Tf_now = Tf(t)
    Tb_now = Tb(t)
    Tf_i_now = Tf_now + q * H / (2 * c_f * rho_f * V_f)
    Tf_o_now = Tf_now - q * H / (2 * c_f * rho_f * V_f)

    # GHE 엑서지
    X_in_ghe_now = (1 - cu.C2K(T_0) / cu.C2K(Tb_now)) * (- q * H) + E_pmp
    X_out_ghe_now = (
        c_f * rho_f * V_f *
        ((cu.C2K(Tf_o_now) - cu.C2K(T_0)) -
         cu.C2K(T_0) * np.log(cu.C2K(Tf_o_now) / cu.C2K(T_0)))
        -
        c_f * rho_f * V_f *
        ((cu.C2K(Tf_i_now) - cu.C2K(T_0)) -
         cu.C2K(T_0) * np.log(cu.C2K(Tf_i_now) / cu.C2K(T_0)))
    )
    X_c_ghe_now = X_in_ghe_now - X_out_ghe_now

    # Ground 엑서지
    X_in_ground_now = (1 - cu.C2K(T_0) / cu.C2K(T_g)) * (- q * H)
    X_out_ground_now = (1 - cu.C2K(T_0) / cu.C2K(Tb_now)) * (- q * H)
    X_c_ground_now = X_in_ground_now - X_out_ground_now

    return [
        R_g, Tf_i_now, Tf_o_now, Tf_now, Tb_now,
        X_in_ghe_now, X_out_ghe_now, X_c_ghe_now,
        X_in_ground_now, X_out_ground_now, X_c_ground_now
    ]

# --- 병렬 실행 ---
with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(executor.map(compute_at_time, times_sec))

for res in results:
    (R_g, Tf_i, Tf_o, Tf_val, Tb_val, Xin_g, Xout_g, Xc_g,
     Xin_gr, Xout_gr, Xc_gr) = res

    R_g_list.append(R_g)
    Tf_i_list.append(Tf_i)
    Tf_o_list.append(Tf_o)
    Tf_list.append(Tf_val)
    Tb_list.append(Tb_val)
    X_in_ghe.append(Xin_g)
    X_out_ghe.append(Xout_g)
    X_c_ghe.append(Xc_g)
    X_in_ground.append(Xin_gr)
    X_out_ground.append(Xout_gr)
    X_c_ground.append(Xc_gr)

# --- DataFrame 생성 및 저장 ---
df = pd.DataFrame({
    'time_hour': times_hour,
    'R_g': R_g_list,
    'Tf_i': Tf_i_list,
    'Tf_o': Tf_o_list,
    'Tf': Tf_list,
    'Tb': Tb_list,
    'X_in_ghe': X_in_ghe,
    'X_out_ghe': X_out_ghe,
    'X_c_ghe': X_c_ghe,
    'X_in_ground': X_in_ground,
    'X_out_ground': X_out_ground,
    'X_c_ground': X_c_ground,
})

df.to_csv('data/GHE_exergy.csv', index=False)
print("CSV 저장 완료.")


Traceback (most recent call last):
  File "C:\Users\pshps\AppData\Roaming\uv\python\cpython-3.11.11-windows-x86_64-none\Lib\multiprocessing\queues.py", line 246, in _feed
    send_bytes(obj)
  File "C:\Users\pshps\AppData\Roaming\uv\python\cpython-3.11.11-windows-x86_64-none\Lib\multiprocessing\connection.py", line 184, in send_bytes
    self._check_closed()
  File "C:\Users\pshps\AppData\Roaming\uv\python\cpython-3.11.11-windows-x86_64-none\Lib\multiprocessing\connection.py", line 137, in _check_closed
    raise OSError("handle is closed")
OSError: handle is closed
Traceback (most recent call last):
  File "C:\Users\pshps\AppData\Roaming\uv\python\cpython-3.11.11-windows-x86_64-none\Lib\multiprocessing\queues.py", line 246, in _feed
    send_bytes(obj)
  File "C:\Users\pshps\AppData\Roaming\uv\python\cpython-3.11.11-windows-x86_64-none\Lib\multiprocessing\connection.py", line 184, in send_bytes
    self._check_closed()
  File "C:\Users\pshps\AppData\Roaming\uv\python\cpython-3.11.11-w

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

cvs 파일 불러오기

In [6]:
data = pd.read_csv('data/GHE_exergy.csv')

times_hour = np.logspace(0, 5, 200)

R_g_list = data['R_g'] 

Tf_i_list = data['Tf_i']
Tf_o_list = data['Tf_o']
Tf_list = data['Tf']
Tb_list = data['Tb']

X_in_ghe = data['X_in_ghe']
X_c_ghe = data['X_c_ghe']
X_out_ghe = data['X_out_ghe']

X_in_ground = data['X_in_ground']
X_c_ground = data['X_c_ground']
X_out_ground = data['X_out_ground']

In [22]:
X_out_ground.tail()

195    60.775387
196    59.960898
197    59.151299
198    58.346719
199    57.547295
Name: X_out_ground, dtype: float64

In [24]:
1 - 57.547295 / E_pmp

0.712263525

# 서브 시스템별 플랏

In [11]:
# Set up the plot
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(dm.cm2in(17), dm.cm2in(7)))
plt.subplots_adjust(left=0.08, right=0.98, top=0.91, bottom=0.17, wspace=0.15)

# Fontsize
label_fontsize = dm.fs(1.5)
tick_fontsize = dm.fs(1)
legend_fontsize = dm.fs(-0.5)
subtitle_fontsize = dm.fs(0.5)

# Common setting
label_pad = 5
line_width = 1

# Color
colors = ['dm.apple green', 'dm.silver', 'dm.azure']

# Plot for the first graph
ax = axes[0]
ax.set_xscale('log')
ax.plot(times_hour, X_in_ground, color=colors[0], label='Exergy input ($X_g$)', linewidth=line_width)
ax.fill_between(times_hour, X_out_ground, X_in_ground, where=(np.array(X_in_ground) > np.array(X_out_ground)),
                color=colors[1], alpha=0.2, label='Exergy consumption ($X_{c,g}$)')
ax.plot(times_hour, X_out_ground, color=colors[2], label='Exergy output ($X_b$)', linewidth=line_width)

# Label
ax.xaxis.set_major_locator(LogLocator(base=10.0, numticks=6))
ax.xaxis.set_major_formatter(LogFormatterMathtext())
ax.set_xlim(1e0, 1e5)
ax.set_xlabel('Hour [h]', fontsize=label_fontsize, labelpad=label_pad)
ax.set_ylabel('Exergy [W]', fontsize=label_fontsize, labelpad=label_pad)


# Limit
ax.set_ylim(-10, 500)

# Tick
ax.set_yticks(np.arange(0, 501, 100))
ax.set_yticklabels(np.arange(0, 501, 100), fontsize=tick_fontsize)
ax.tick_params(axis='both', which='major', labelsize=tick_fontsize)
ax.text(0.02, 1.09, '(a) Ground', transform=ax.transAxes, fontsize=subtitle_fontsize, fontweight='medium', va='top', ha='left')

# Grid
ax.grid(True)

# Legend
ax.legend(
    loc='upper right', ncol=1, frameon=False,
    fontsize=legend_fontsize, fancybox=False,
    columnspacing=1, labelspacing=0.6,
    bbox_to_anchor=(0.99, 0.99),
    handlelength=1.8
)

# Plot for the second graph
ax = axes[1]
ax.set_xscale('log')
ax.plot(times_hour, X_in_ghe, color=colors[0], label='Exergy input ($X_b + E_{pmp}$)', linewidth=line_width)
ax.fill_between(times_hour, X_out_ghe, X_in_ghe, where=(np.array(X_in_ghe) > np.array(X_out_ghe)),
                color=colors[1], alpha=0.2, label='Exergy consumption ($X_{c,GHE}$)')
ax.plot(times_hour, X_out_ghe, color=colors[2], label='Exergy output ($X_{f,o}-X_{f,i}$)', linewidth=line_width)

# Label
ax.xaxis.set_major_locator(LogLocator(base=10.0, numticks=6))
ax.xaxis.set_major_formatter(LogFormatterMathtext())
ax.set_xlim(1e0, 1e5)
ax.set_xlabel('Hour [h]', fontsize=label_fontsize, labelpad=label_pad)

# Limit
ax.set_ylim(-10, 500)

# Tick
ax.set_yticks(np.arange(0, 501, 100))
ax.set_yticklabels(np.arange(0, 501, 100), fontsize=tick_fontsize)
ax.tick_params(axis='both', which='major', labelsize=tick_fontsize)
ax.text(0.02, 1.09, '(b) Ground heat exchanger', transform=ax.transAxes, fontsize=subtitle_fontsize, fontweight='medium', va='top', ha='left')

# Grid
ax.grid(True)

# Legend
ax.legend(
    loc='upper right', ncol=1, frameon=False,
    fontsize=legend_fontsize, fancybox=False,
    columnspacing=1, labelspacing=0.6,
    bbox_to_anchor=(0.99, 0.99),
    handlelength=1.8
)

# Save and show the figure
plt.savefig('figure/exergy_subsystem.png', dpi=600)
dm.util.save_and_show(fig)

# 온도 플랏

In [25]:
# Set up the plot
fig, ax = plt.subplots(figsize=(dm.cm2in(14), dm.cm2in(7)))
plt.subplots_adjust(left=0.1, right=0.95, top=0.91, bottom=0.17)

# Fontsize
label_fontsize = dm.fs(2)
tick_fontsize = dm.fs(1.5)
legend_fontsize = dm.fs(-0.5)

# Common setting
label_pad = 5
line_width = 0.7

# Color
colors = ['dm.lime4', 'dm.purply', 'dm.soft blue', 'dm.mango']

# Plot
ax.set_xscale('log')
ax.plot(times_hour, Tf_i_list, color=colors[0], label='Inlet fluid temperature', linewidth=line_width)
ax.plot(times_hour, Tf_list, color=colors[1], label='Mean fluid temperature', linewidth=line_width)
ax.plot(times_hour, Tf_o_list, color=colors[2], label='Outlet fluid temperature', linewidth=line_width)
ax.plot(times_hour, Tb_list, color=colors[3], label='Borehole wall temperature', linewidth=line_width)

# Label
ax.xaxis.set_major_locator(LogLocator(base=10.0, numticks=6))
ax.xaxis.set_major_formatter(LogFormatterMathtext())
ax.set_xlim(1e0, 1e5)
ax.set_xlabel('Hour [h]', fontsize=label_fontsize, labelpad=label_pad)
ax.set_ylabel('Temperature [ºC]', fontsize=label_fontsize, labelpad=label_pad)

# Limit
ax.set_ylim(14, 35)

# Tick
ax.set_yticks(np.arange(15, 36, 5))
ax.set_yticklabels(np.arange(15, 36, 5), fontsize=tick_fontsize)
ax.tick_params(axis='both', which='major', labelsize=tick_fontsize)
ax.tick_params(axis='both', which='minor', labelsize=tick_fontsize)

# Grid
ax.grid(True)

# Legend
ax.legend(
    loc='upper right', ncol=1, frameon=False,
    fontsize=legend_fontsize, fancybox=False,
    columnspacing=0.8, labelspacing=0.8,
    bbox_to_anchor=(0.99, 0.38),
    handlelength=1.8
)

plt.savefig('figure/temperature.png', dpi=600)
dm.util.save_and_show(fig)

# 토양 열저항 플랏

In [26]:
# Set up the plot
fig, ax = plt.subplots(figsize=(dm.cm2in(14), dm.cm2in(7)))
plt.subplots_adjust(left=0.13, right=0.95, top=0.91, bottom=0.2)

# Fontsize
label_fontsize = dm.fs(2)
tick_fontsize = dm.fs(1.5)
legend_fontsize = dm.fs(-0.5)

# Common setting
label_pad = 7
line_width = 0.7

# Color
colors = ['dm.red4']

# Plot
ax.set_xscale('log')
ax.plot(times_hour, R_g_list, color=colors[0], label='Ground thermal resistance', linewidth=line_width)

# Label
ax.xaxis.set_major_locator(LogLocator(base=10.0, numticks=6))
ax.xaxis.set_major_formatter(LogFormatterMathtext())
ax.set_xlim(1e0, 1e5)
ax.set_xlabel('Hour [h]', fontsize=label_fontsize, labelpad=label_pad)
ax.set_ylabel('$R_g(t)$ [(m·K)/W]', fontsize=label_fontsize, labelpad=label_pad)

# Limit
ax.set_ylim(-0.02, 0.5)

# Tick
ax.set_yticks(np.arange(0, 0.6, 0.1))
ax.tick_params(axis='both', which='major', labelsize=tick_fontsize)
ax.tick_params(axis='both', which='minor', labelsize=tick_fontsize)

# Grid
ax.grid(True)

plt.savefig('figure/Ground thermal resistance.png', dpi=600)
dm.util.save_and_show(fig)